In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from datasets import load_dataset

In [ ]:
df = pd.read_csv("/kaggle/input/alzheimers-disease-risk-prediction-eu-business/train.csv")
df.head(5)
print(f"The number of columns/features: {df.shape[1]}")
print(f"The number of null:\n{df.isnull().sum()}")

In [ ]:
data = df.iloc[:, 1:35]
data.head(5)
print(data["DoctorInCharge"].unique())

In [ ]:
# Omit "DoctorInCharge" from list of features for data analysis
data = df.iloc[:, 1:34]
data.head(5)

1. Nominal Categorical - Matthews Correlation Coefficient
    1) Gender
    2) Ethnicity
    3) Smoking
    4) FamilyHistoryAlzheimers,
    5) CardiovascularDisease,
    6) Diabetes,
    7) Depression,
    8) HeadInjury,
    9) Hypertension,
    10) MemoryComplaints
    11) BehavioralProblems
    12) Confusion
    13) Disorientation
    14) PersonalityChanges
    15) DifficultyCompletingTasks
    16) Forgetfulness
    
2. Ordinal Categorical - Cramér's V
    1) Age,
    2) AlcoholConsumption,
    3) PhysicalActivity,
    4) DietQuality,
    5) SleepQuality,
    6) MMSE: Mini-Mental State Exam score (0–30).
    7) FunctionalAssessment: Functional score (0–10).
    8) ADL: Activities of Daily Living score (0–10).
       
3. Continuous Numerical - Point-biseral Correlation
    1) BMI
    2) SystolicBP
    3) DiastolicBP
    4) CholesterolTotal
    5) CholesterolLDL
    6) CholesterolHDL
    7) CholesterolTriglycerides

4. Diagnosis - Nominal Categorical

In [ ]:
data_corr = data.corr(method="pearson")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20, 20)

mask = np.triu(np.ones_like(data_corr))
ax = sns.heatmap(data_corr, annot=True, cmap="coolwarm", vmax=1, vmin=-1, mask=mask, fmt=".2f")

#### Features highly related to Diagnosis
1. FunctionalAssessment(-0.38)
2. ADL(-0.34)
3. MemoryComplaints(0.3)
4. MMSE(-0.22)
5. BehavioralProblems(0.21)

Unexpectedly, "Age" is not related to Alzheimer Diagonis. So we have to check distribution of values in Age

In [ ]:
df["Age"].unique()

In [ ]:
##### Set up Diagnosis as "y_train"
y_train = df.iloc[:, 33]
y_train.head(5)